In [1]:
#Imports
import pandas as pd
from datetime import datetime
import numpy as np

#Variables
print("How many players?")
players = int(input())
print("What is the month and year? (MMM/YYYY)")
monthAndYear = input()
print("What is the in-game date? (DD/MM/YYYY)")
gameDate = input()

#DataFrame Settings
df = pd.read_excel("HJK {monthAndYear}.xlsx".format(monthAndYear = monthAndYear))
pd.set_option('display.max_rows', 100)


#Constants
month = 31
year = 365

#Program
pos = df['Best Pos']
df.drop(['Best Pos', "No.", "Nat", "Ability", "Potential"], axis=1,inplace = True)
df.insert(0, 'Best Pos', pos)

df.loc[df['Name'].str.split().str.len() == 3, 'Name'] = df['Name'].str.split().str[-1]
df.loc[df['Name'].str.split().str.len() == 2, 'Name'] = df['Name'].str.split().str[-1]
df.loc[df['Name'].str.split().str.len() == 1, 'Name'] = df['Name'].str.split().str[-1]

df["Best Pos"].replace("D (R)", "RB", inplace=True)
df["Best Pos"].replace("D (C)", "CB", inplace=True)
df["Best Pos"].replace("D (L)", "LB", inplace=True)

df["Best Pos"].replace("M (R)", "RM", inplace=True)
df["Best Pos"].replace("M (C)", "CM", inplace=True)
df["Best Pos"].replace("M (L)", "LM", inplace=True)

df["Best Pos"].replace("AM (R)", "RW", inplace=True)
df["Best Pos"].replace("AM (C)", "AM", inplace=True)
df["Best Pos"].replace("AM (L)", "LW", inplace=True)

df["Best Pos"].replace("ST (C)", "ST", inplace=True)

df["Home-Grown Status"].replace("Trained at club (15-21)", "C", inplace=True)
df["Home-Grown Status"].replace("Trained at club (0-21)", "C", inplace=True)
df["Home-Grown Status"].replace("Trained in nation (15-21)", "H", inplace=True)
df["Home-Grown Status"].replace("Trained in nation (0-21)", "H", inplace=True)
#df["Home-Grown Status"].replace("-", "", inplace=True)
#df["Due Date"].replace("club (15-21)", "C", regex=True, inplace=True)
df["Due Date"].replace("\(club 15-21\)", "C", regex=True, inplace=True)
df["Due Date"].replace("\(nation 15-21\)", "H", regex=True, inplace=True)


df['Date'], df['Future State'] = df['Due Date'].str.split(' ', 1).str
df['Future State'] = df['Future State'].fillna("-")
df.drop(["Due Date"], axis=1,inplace = True)

df.insert(7, "Actual Date", [gameDate] * players)

df[["Actual Date",'Date']] = df[["Actual Date",'Date']].apply(pd.to_datetime, errors="coerce")


df['HGN Days'] = (df['Date'] - df['Actual Date']).dt.days
df["HGN Days"] = df["HGN Days"].fillna(0)
df.drop(["Actual Date", "Date"], axis=1,inplace = True)
df["HGN Days"] = (df["HGN Days"] / month).apply(np.ceil)

df["HGN Days"] = df["HGN Days"].astype(int)

pos = df["Home-Grown Status"]
df.drop(["Home-Grown Status"], axis=1,inplace = True)
df.insert(1, "Home-Grown Status", pos)

df['Transfer Value'], df['Maximum Value'] = df['Transfer Value'].str.split(' - ', 1).str
df.drop(["Transfer Value"], axis=1,inplace = True)
df["Maximum Value"] = df["Maximum Value"].fillna(0)

df["Home-Grown Status"].replace("-", "", regex=True, inplace=True)
df["Future State"].replace("-", "", regex=True, inplace=True)

df.rename(columns = {'Home-Grown Status':'Status', 'Future State':'Fut', "HGN Days":"Months"}, inplace = True)

pos = df["Fut"]
df.drop(["Fut"], axis=1,inplace = True)
df.insert(2, "Fut", pos)

df["Maximum Value"] = df["Maximum Value"].str.replace('£', '').str.replace('K', '000').str.replace('M', '00000').str.replace(".", "")
df["Maximum Value"] = df["Maximum Value"].fillna(0)
df["Maximum Value"] = df["Maximum Value"].astype(int)

#df = df.sort_values(by=['Maximum Value'], ascending=False)

df["HGN"] = df["Status"] + df["Fut"]
df.drop(["Maximum Value", "Status", "Fut"], axis=1,inplace = True)


pos = df['HGN']
df.drop(['HGN'], axis=1,inplace = True)
df.insert(0, 'HGN', pos)

pos = df['Months']
df.drop(['Months'], axis=1,inplace = True)
df.insert(1, 'Months', pos)

mask = df['Best Pos'].ne(df['Best Pos'].shift(-1))
df1 = pd.DataFrame('',index=mask.index[mask] + .5, columns=df.columns)
df = pd.concat([df, df1]).sort_index().reset_index(drop=True).iloc[:-1]

df.drop(['Best Pos'], axis=1,inplace = True)
df["Months"].replace(0, "", inplace=True)


df.to_excel('HJK {monthAndYear} Final.xlsx'.format(monthAndYear = monthAndYear), index=False, startrow=-1)



df


How many players?
76
What is the month and year? (MMM/YYYY)
Aug 2027
What is the in-game date? (DD/MM/YYYY)
15/08/2027


<ipython-input-1-48156fe83522>:56: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Date'], df['Future State'] = df['Due Date'].str.split(' ', 1).str
<ipython-input-1-48156fe83522>:76: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Transfer Value'], df['Maximum Value'] = df['Transfer Value'].str.split(' - ', 1).str
<ipython-input-1-48156fe83522>:89: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df["Maximum Value"] = df["Maximum Value"].str.replace('£', '').str.replace('K', '000').str.replace('M', '00000').str.replace(".", "")


,HGN,Months,Name,Age
0,C,,Høyenhall,21
1,C,,Soisalo,19
2,,,Fernández,20
3,C,37,Jiménez,18
4,C,,Kavén,19
5,C,35,Hautala,18
6,,,,
7,H,,Haapasalo,21
8,C,,Aðalgeirsson,19
9,,,Ng,31


In [56]:
df["Best Pos"].unique()

array(['GK', 'RB', 'DM', 'RM', 'CM', 'LM', 'RW', 'AM', 'LW', 'ST'],
      dtype=object)

In [122]:
round(0.51)

1

In [180]:
#Imports
import pandas as pd
from datetime import datetime
import numpy as np

#DataFrame Settings
df = pd.read_excel("HJK Nov 2022.xlsx")
pd.set_option('display.max_rows', 100)

#Constants
month = 31
year = 365

#Program
pos = df['Best Pos']
df.drop(['Best Pos', "No.", "Nat", "Ability", "Potential"], axis=1,inplace = True)
df.insert(0, 'Best Pos', pos)

df.loc[df['Name'].str.split().str.len() == 3, 'Name'] = df['Name'].str.split().str[-1]
df.loc[df['Name'].str.split().str.len() == 2, 'Name'] = df['Name'].str.split().str[-1]
df.loc[df['Name'].str.split().str.len() == 1, 'Name'] = df['Name'].str.split().str[-1]

df["Best Pos"].replace("D (R)", "RB", inplace=True)
df["Best Pos"].replace("D (C)", "RB", inplace=True)
df["Best Pos"].replace("D (L)", "RB", inplace=True)

df["Best Pos"].replace("M (R)", "RM", inplace=True)
df["Best Pos"].replace("M (C)", "CM", inplace=True)
df["Best Pos"].replace("M (L)", "LM", inplace=True)

df["Best Pos"].replace("AM (R)", "RW", inplace=True)
df["Best Pos"].replace("AM (C)", "AM", inplace=True)
df["Best Pos"].replace("AM (L)", "LW", inplace=True)

df["Best Pos"].replace("ST (C)", "ST", inplace=True)

df["Home-Grown Status"].replace("Trained at club (15-21)", "C", inplace=True)
df["Home-Grown Status"].replace("Trained at club (0-21)", "C", inplace=True)
df["Home-Grown Status"].replace("Trained in nation (15-21)", "H", inplace=True)
df["Home-Grown Status"].replace("Trained in nation (0-21)", "H", inplace=True)
#df["Home-Grown Status"].replace("-", "", inplace=True)
#df["Due Date"].replace("club (15-21)", "C", regex=True, inplace=True)
df["Due Date"].replace("\(club 15-21\)", "C", regex=True, inplace=True)
df["Due Date"].replace("\(nation 15-21\)", "H", regex=True, inplace=True)


df['Date'], df['Future State'] = df['Due Date'].str.split(' ', 1).str
df['Future State'] = df['Future State'].fillna("-")
df.drop(["Due Date"], axis=1,inplace = True)


df[["Actual Date",'Date']] = df[["Actual Date",'Date']].apply(pd.to_datetime, errors="coerce")

df

<ipython-input-180-e1ce44d319f3>:47: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Date'], df['Future State'] = df['Due Date'].str.split(' ', 1).str


,Best Pos,Name,Age,Transfer Value,Home-Grown Status,Actual Date,Date,Future State
0,GK,Fabiański,37,£14K - £60K,-,2022-11-29,NaT,-
1,GK,Sinisalo,21,£120K - £500K,-,2022-11-29,NaT,-
2,GK,Tånnander,22,£170K - £425K,-,2022-11-29,NaT,-
3,GK,Henriksson,19,£24K - £200K,C,2022-11-29,NaT,-
4,GK,Partanen,18,£3.5K - £26K,H,2022-11-29,2024-11-28,C
5,GK,Ramula,17,£2K - £120K,C,2022-11-29,2022-12-29,H
6,GK,Jantunen,17,£2.5K - £55K,C,2022-11-29,2022-12-29,C
7,GK,Jadue,17,£6K - £230K,H,2022-11-29,2022-12-30,C
8,RB,Gbé,20,£35K - £190K,-,2022-11-29,2023-02-20,H
9,RB,Moren,31,£220K - £350K,C,2022-11-29,NaT,-


In [202]:
#Imports
import pandas as pd
from datetime import datetime
import numpy as np

#DataFrame Settings
df = pd.read_excel("HJK Nov 2022.xlsx")
pd.set_option('display.max_rows', 100)

df.insert(8, "Actual Date", ["29/11/2022"] * 77)

df

,No.,Name,Age,Best Pos,Nat,Transfer Value,Home-Grown Status,Due Date,Actual Date,Ability,Potential
0,-,Łukasz Fabiański,37,GK,POL,£14K - £55K,-,-,29/11/2022,NaN,NaN
1,35,Viljami Sinisalo,21,GK,FIN,£120K - £500K,-,-,29/11/2022,NaN,NaN
2,25,Jakob Tånnander,22,GK,SWE,£170K - £425K,-,-,29/11/2022,NaN,NaN
3,-,Elmo Henriksson,19,GK,FIN,£24K - £200K,Trained at club (15-21),-,29/11/2022,NaN,NaN
4,-,Aleksi Partanen,18,GK,FIN,£3.5K - £26K,Trained in nation (0-21),5/12/2024 (club 15-21),29/11/2022,NaN,NaN
5,-,Alex Ramula,17,GK,FIN,£2K - £120K,Trained at club (0-21),29/12/2022 (nation 15-21),29/11/2022,NaN,NaN
6,-,Jaaso Jantunen,17,GK,FIN,£2.5K - £55K,Trained at club (0-21),29/12/2022 (club 15-21),29/11/2022,NaN,NaN
7,-,Axel Jadue,17,GK,CHI,£6K - £230K,Trained in nation (15-21),30/12/2022 (club 15-21),29/11/2022,NaN,NaN
8,-,Jean-Jaurés Gbé,20,D (R),CIV,£65K - £375K,-,20/2/2023 (nation 15-21),29/11/2022,NaN,NaN
9,15,Valtteri Moren,31,D (R),FIN,£220K - £350K,Trained at club (15-21),-,29/11/2022,NaN,NaN


In [10]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
import cv2

ModuleNotFoundError: No module named 'cv2'